In [6]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-wikipedia-qa/resolve/main/wikipedia-qa.jsonl

In [14]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [8]:
data = []
with open('wikipedia-qa.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [9]:
len(data)

44923

In [10]:
indices = {i for i in range(len(data))}

In [19]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data))):
    query = data[i]['paragraph']
    
    for s in data[i]['qa']['qa']:
        pos = [query]
        sampled = random.sample(list(indices - {i}), 100)
        neg = []
        for k in sampled:
            try:
                overlapped = overlap(query, data[k]['paragraph'])
                if overlapped < 0.3:
                    neg.append(data[k]['paragraph'])
                if len(neg) >= 5:
                    break
            except:
                pass
            
        pairs.append({
            'query': s['question'],
            'pos': pos,
            'neg': neg
        })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44923/44923 [27:52<00:00, 26.86it/s]


In [20]:
len(pairs)

329021

In [21]:
with open('chatgpt-wikipedia-qa-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [22]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='chatgpt-wikipedia-qa-pair.jsonl',
    path_in_repo='chatgpt-wikipedia-qa-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

chatgpt-wikipedia-qa-pair.jsonl:   0%|          | 0.00/7.53G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/85c0df57d5199beeb2962a2c0f4adc3d75964d08', commit_message='Upload chatgpt-wikipedia-qa-pair.jsonl with huggingface_hub', commit_description='', oid='85c0df57d5199beeb2962a2c0f4adc3d75964d08', pr_url=None, pr_revision=None, pr_num=None)